In [1]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
from private_functions import getAPI

Here are my early helper functions I set up for ease of access to lol_watcher and its uses. 

In [2]:
def get_puuid(name):
    """
    Input is summoner name. Outputs players puuid.
    """
    return lol_watcher.summoner.by_name(region, name)['puuid']
def get_matchlist(name):
    """
    Input is summoner name. Output is a list of the 100 most recent ranked matches for that summoner.
    """
    return lol_watcher.match.matchlist_by_puuid(region, get_puuid(name), count=100, type='ranked')
def get_match(match_id):
    '''
    Helper function that allows every other get function to not need to call the lol_watcher everytime it runs.
    '''
    return lol_watcher.match.by_id(region, match_id)

Do not ever put API's in your repositories. I have mine in a local file and call it here.

I also have my region defaulted to 'na1' and my match_id set to one of my own personal games for testing. 

In [3]:
myApi = getAPI()
lol_watcher = LolWatcher(myApi)

In [4]:
region = 'na1'
match_id = 'NA1_4286956599'
match = get_match(match_id)

Here is my first look into what I will be working with. It is how a match comes straight from the API without any cleaning or tinkering. At first glance it is a bit overwhelming, but with patenince I can help break it down.

In [5]:
match

{'metadata': {'dataVersion': '2',
  'matchId': 'NA1_4286956599',
  'participants': ['oOUDXEZnLJdcXhLQlPD9hPLgd0k0BKde7xFhcogb7-xNvQ--M_ZOIM94-B9Ji_J0qM8i48D1vmvVwQ',
   'JgFpRq3Leoe9Mn_GzPSvCholBc7VVqNYh6x8hKfUe--JH4NgKuEIufzc3xpV58DWkzw9uA8WHMaeHw',
   '4Yfk03L9agtZMzJU-RENCM_XDZfkkMmZhyH0Hcc294Bxib5KEAooyLQ_hmu0VAHwgHG8fPxi1vRGew',
   'LqVxfFFNqKspmB2GLnppd6SnVZTgdlrxep_2XqwqJMMXnHbC2ZzbnIVTX0wF-l8THeH_kQ0uSceo0w',
   'Y5S57zDapZMqHE4mI3-NstSAPi5kDfIb3UDsqIhDF-ZCRwhAu5Dl63vAKyp_nfx87X5SMI9gPhRAhQ',
   'feMIK6EL0c3sg6Ytx83CnZxLWsZq5ywbkiv4IrVzYFVr-bxtX-ncxHtCd5NOtg5Gk-75XsWVj_9IwA',
   '-SaEIp4mr8jQKR4z2yWmOr2vWwMbeu5CTKOZygxmyj2TPId7sHfhqHVoIPdtH3UPdm5aN0aYrgOmnw',
   'AFqllfLV1CKJDzemzngWK5n7Kq93tNGqxOLKiBjAGbVNtHbAg4DT84DLy9pMVa9LciZrTnE8-dZtDg',
   'leqQwaP8IcmQXBtOKDGMeXfpxYzHqgXIsT4VzsaCve9xH3o2X2BB88LEcU9yA4Z8a1O7HmSOIlM_uw',
   'lGNgnHZQRfEk2xZ8Q4gEF9dmZ7bNn8t-pjku4krrIXD87taW0A_0Wn5hZKw8J3pDY-ART1O4IWKhJg']},
 'info': {'gameCreation': 1650778290000,
  'gameDuration': 1905,
  

Here are the functions that make up the main part of my pipeline. The goal is to clean and consolidate each match into a single row for a Pandas DataFrame. I explain how I came up with these lower if you are interested. 

In [57]:
def get_duration(match):
    '''
    Uses a match from get_match to find the games duration and returns a tiny dictionary. 
    '''
    return {'gameDuration' : match['info']['gameDuration']}

def get_win(match):
    '''
    Uses a match from get_match to return whether or not the team we are analyzing won. 
    '''
    return {'win' : match['info']['teams'][0]['win']}

def get_team_objectives(match):
    '''
    Uses a match from get_match to flatten a teams objectives for use in a DataFrame.
    Returns a dictionary of team objectives.
    '''
    obj = match['info']['teams'][0]['objectives']
    new = {}
    for k in obj:
        for key in obj[k]:
            new[k+'_'+key] = obj[k][key]
    return new

def get_avgs(match):
    '''
    Uses a match from get_match to consolidate the 5 participants from the team we are analyzing to one average.
    Returns a dictionary of averaged out statistics pertaining to team 1. 
    '''
    avg_keys = ['kills', 'deaths', 'assists', 'champLevel', 'champExperience', 'doubleKills', 'tripleKills', 'quadraKills', 
            'pentaKills','consumablesPurchased', 'damageDealtToBuildings',
            'damageDealtToObjectives', 'damageSelfMitigated', 'detectorWardsPlaced', 'timePlayed',
            'goldEarned', 'goldSpent', 'inhibitorKills', 'itemsPurchased', 'largestKillingSpree', 'killingSprees',
            'longestTimeSpentLiving', 'magicDamageDealt','magicDamageDealtToChampions', 'magicDamageTaken',
            'physicalDamageDealt', 'physicalDamageDealtToChampions', 'physicalDamageTaken', 'totalDamageDealt',
            'totalDamageDealtToChampions', 'totalDamageShieldedOnTeammates', 'totalDamageTaken', 
            'trueDamageDealt', 'trueDamageDealtToChampions', 'trueDamageTaken',
            'totalHeal', 'totalHealsOnTeammates', 'totalMinionsKilled', 'totalTimeCCDealt', 'totalTimeSpentDead',
            'turretKills', 'turretsLost', 'neutralMinionsKilled', 'objectivesStolen', 'objectivesStolenAssists', 
            'visionScore','visionWardsBoughtInGame', 'wardsKilled', 'wardsPlaced']
    new = {}
    for i in match['info']['participants']:
        if i['teamId'] == 100:
            for j in avg_keys:
                try:
                    new['avg'+j[0].upper()+j[1:]] += i[j]
                except:
                    new['avg'+j[0].upper()+j[1:]] = i[j]
    new.update((k, round(v * 0.2, 3)) for k,v in new.items())
    return new

def get_challenges(match):
    '''
     Uses a match from get_match to consolidate the 5 participants challenges and average them out.
     Returns a dictionary of all the challanges averages for team 1. 
    '''
    chal_keys = ['abilityUses', 'acesBefore15Minutes', 'alliedJungleMonsterKills', 'baronTakedowns', 
                 'blastConeOppositeOpponentCount', 'bountyGold', 'buffsStolen', 'damagePerMinute', 
                 'deathsByEnemyChamps', 'dragonTakedowns', 
                 'effectiveHealAndShielding',
                 'enemyChampionImmobilizations', 'enemyJungleMonsterKills', 'epicMonsterSteals',
                 'goldPerMinute','jungleCsBefore10Minutes', 'kda', 'killParticipation', 'killsNearEnemyTurret',
                 'killsUnderOwnTurret', 'killsWithHelpFromEpicMonster',
                 'maxKillDeficit', 'multikills', 'outnumberedKills', 
                 'riftHeraldTakedowns','scuttleCrabKills', 'skillshotsDodged', 
                 'skillshotsHit', 'soloKills', 'takedowns',
                 'takedownsAfterGainingLevelAdvantage', 'takedownsBeforeJungleMinionSpawn', 
                 'turretPlatesTaken', 'turretTakedowns',
                 'turretsTakenWithRiftHerald', 'visionScorePerMinute',
                 'wardTakedownsBefore20M', 'wardsGuarded']
    new = {}
    for i in match['info']['participants']:
        if i['teamId'] == 100:
            for j in chal_keys:
                    try:
                        new['avg'+j[0].upper()+j[1:]] += i['challenges'][j]
                    except:
                        new['avg'+j[0].upper()+j[1:]] = i['challenges'][j]
    new.update((k, round(v * 0.2, 4)) for k,v in new.items())
    return new

def get_row(match_id):
    ''' 
    Puts everything together into a single row for a new DataFrame
    '''
    match = get_match(match_id)
    data = {}
    data.update(get_avgs(match))
    data.update(get_challenges(match))
    data.update(get_team_objectives(match))
    data.update(get_duration(match))
    data.update(get_win(match))
    return data

This is a list of my friends and their summoner names. I plan on using this to generate games to analyze. But for now, its useless.

In [49]:
summoners = ['Zealfire', 'SmurfHD', 'DumbAmerican', 'About 10 Inches', 'Exprience', 'Leaders1', 
             'Kehlee', 'KybitOW', 'DodoMuncher', 'Mango Bleach']

This is a first glance and things get a little hairer so stick with me. If we take a look at the keys to a match, we only get `'metadata'` and `'info'`. So lets take a look. 

In [9]:
match.keys()

dict_keys(['metadata', 'info'])

As you can see there isn't a whole lost of useful information in the metadata. It tell us the matchId, which is the region and gameId combined. And it tell us the puuid's of the summoners who are playing. 

Info on the other hand, looks like it has all the information we are going to need. And is quite large and therefore harder to understand. 

In [10]:
match['metadata'], match['info']

({'dataVersion': '2',
  'matchId': 'NA1_4286956599',
  'participants': ['oOUDXEZnLJdcXhLQlPD9hPLgd0k0BKde7xFhcogb7-xNvQ--M_ZOIM94-B9Ji_J0qM8i48D1vmvVwQ',
   'JgFpRq3Leoe9Mn_GzPSvCholBc7VVqNYh6x8hKfUe--JH4NgKuEIufzc3xpV58DWkzw9uA8WHMaeHw',
   '4Yfk03L9agtZMzJU-RENCM_XDZfkkMmZhyH0Hcc294Bxib5KEAooyLQ_hmu0VAHwgHG8fPxi1vRGew',
   'LqVxfFFNqKspmB2GLnppd6SnVZTgdlrxep_2XqwqJMMXnHbC2ZzbnIVTX0wF-l8THeH_kQ0uSceo0w',
   'Y5S57zDapZMqHE4mI3-NstSAPi5kDfIb3UDsqIhDF-ZCRwhAu5Dl63vAKyp_nfx87X5SMI9gPhRAhQ',
   'feMIK6EL0c3sg6Ytx83CnZxLWsZq5ywbkiv4IrVzYFVr-bxtX-ncxHtCd5NOtg5Gk-75XsWVj_9IwA',
   '-SaEIp4mr8jQKR4z2yWmOr2vWwMbeu5CTKOZygxmyj2TPId7sHfhqHVoIPdtH3UPdm5aN0aYrgOmnw',
   'AFqllfLV1CKJDzemzngWK5n7Kq93tNGqxOLKiBjAGbVNtHbAg4DT84DLy9pMVa9LciZrTnE8-dZtDg',
   'leqQwaP8IcmQXBtOKDGMeXfpxYzHqgXIsT4VzsaCve9xH3o2X2BB88LEcU9yA4Z8a1O7HmSOIlM_uw',
   'lGNgnHZQRfEk2xZ8Q4gEF9dmZ7bNn8t-pjku4krrIXD87taW0A_0Wn5hZKw8J3pDY-ART1O4IWKhJg']},
 {'gameCreation': 1650778290000,
  'gameDuration': 1905,
  'gameEndTimestamp': 

I decided to try put the `match['info']` into a Pandas DataFrame to get a better idea of what I am working with. At first glance we can see that there is some useful info all ready for us, for instance gameDuration. So I first went ahead and set up a get function to grab how long the game went for. 

Next I noticed the participants and teams columns have lists of dictionaries and I suspect most of the data is hiding in there. 

In [38]:
pd.DataFrame([match['info']])

,gameCreation,gameDuration,gameEndTimestamp,gameId,gameMode,gameName,gameStartTimestamp,gameType,gameVersion,mapId,participants,platformId,queueId,teams,tournamentCode
0,1650778290000,1905,1650780212165,4286956599,CLASSIC,teambuilder-match-4286956599,1650778306516,MATCHED_GAME,12.7.433.4138,11,"[{'assists': 11, 'baronKills': 0, 'bountyLevel...",NA1,420,"[{'bans': [{'championId': 23, 'pickTurn': 1}, ...",


I checked out teams first. I don't really care about which champion is being banned or when it was banned. But the rest of the info is very important. First I set up another basic get function to grab wether or not they win. And next I check out the objectives. 

In [22]:
match['info']['teams'][0]

{'bans': [{'championId': 23, 'pickTurn': 1},
  {'championId': 131, 'pickTurn': 2},
  {'championId': 111, 'pickTurn': 3},
  {'championId': 498, 'pickTurn': 4},
  {'championId': 157, 'pickTurn': 5}],
 'objectives': {'baron': {'first': True, 'kills': 1},
  'champion': {'first': True, 'kills': 41},
  'dragon': {'first': False, 'kills': 0},
  'inhibitor': {'first': False, 'kills': 0},
  'riftHerald': {'first': True, 'kills': 1},
  'tower': {'first': False, 'kills': 5}},
 'teamId': 100,
 'win': False}

In [23]:
match['info']['teams'][0]['objectives'].keys()

dict_keys(['baron', 'champion', 'dragon', 'inhibitor', 'riftHerald', 'tower'])

As you can see the objectives are very important but are not in a good format that would be easy to analyze. So we have a little work to do. 

In [24]:
obj = match['info']['teams'][0]['objectives']
obj

{'baron': {'first': True, 'kills': 1},
 'champion': {'first': True, 'kills': 41},
 'dragon': {'first': False, 'kills': 0},
 'inhibitor': {'first': False, 'kills': 0},
 'riftHerald': {'first': True, 'kills': 1},
 'tower': {'first': False, 'kills': 5}}

This next cell takes the original objectives and flattens it out to be a single dictionary filled with all the data we needed from this section. This way we can concat this onto a larger dictionary and build a very useful DataFrame with it. You might even recognize this cell as very similar code is now in my pipeline. 

In [25]:
new = {}
for k in obj:
    for key in obj[k]:
        new[k+key.capitalize()] = obj[k][key]
new

{'baronFirst': True,
 'baronKills': 1,
 'championFirst': True,
 'championKills': 41,
 'dragonFirst': False,
 'dragonKills': 0,
 'inhibitorFirst': False,
 'inhibitorKills': 0,
 'riftHeraldFirst': True,
 'riftHeraldKills': 1,
 'towerFirst': False,
 'towerKills': 5}

My next hurdle is a big one. I need to take 5 individual team members and get the average for every continuous variable in participants. First things first, I need to take a good look at what variables are available and slowly pick which ones would be helpful for analysis. I also put the variables in a more intuitive order, similar variables by eachother etc.

In [26]:
match['info']['participants'][0].keys()

dict_keys(['assists', 'baronKills', 'bountyLevel', 'challenges', 'champExperience', 'champLevel', 'championId', 'championName', 'championTransform', 'consumablesPurchased', 'damageDealtToBuildings', 'damageDealtToObjectives', 'damageDealtToTurrets', 'damageSelfMitigated', 'deaths', 'detectorWardsPlaced', 'doubleKills', 'dragonKills', 'eligibleForProgression', 'firstBloodAssist', 'firstBloodKill', 'firstTowerAssist', 'firstTowerKill', 'gameEndedInEarlySurrender', 'gameEndedInSurrender', 'goldEarned', 'goldSpent', 'individualPosition', 'inhibitorKills', 'inhibitorTakedowns', 'inhibitorsLost', 'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6', 'itemsPurchased', 'killingSprees', 'kills', 'lane', 'largestCriticalStrike', 'largestKillingSpree', 'largestMultiKill', 'longestTimeSpentLiving', 'magicDamageDealt', 'magicDamageDealtToChampions', 'magicDamageTaken', 'neutralMinionsKilled', 'nexusKills', 'nexusLost', 'nexusTakedowns', 'objectivesStolen', 'objectivesStolenAssists', 'parti

In [27]:
avg_keys = ['kills', 'deaths', 'assists', 'champLevel', 'champExperience', 'doubleKills', 'tripleKills', 'quadraKills', 
            'pentaKills','consumablesPurchased', 'damageDealtToBuildings',
            'damageDealtToObjectives', 'damageSelfMitigated', 'detectorWardsPlaced', 
            'goldEarned', 'goldSpent', 'inhibitorKills', 'itemsPurchased', 'largestKillingSpree', 'killingSprees',
            'longestTimeSpentLiving', 'magicDamageDealt','magicDamageDealtToChampions', 'magicDamageTaken',
            'physicalDamageDealt', 'physicalDamageDealtToChampions', 'physicalDamageTaken', 'totalDamageDealt',
            'totalDamageDealtToChampions', 'totalDamageShieldedOnTeammates', 'totalDamageTaken', 
            'trueDamageDealt', 'trueDamageDealtToChampions', 'trueDamageTaken',
            'totalHeal', 'totalHealsOnTeammates', 'totalMinionsKilled', 'totalTimeCCDealt', 'totalTimeSpentDead',
            'turretKills', 'turretsLost', 'neutralMinionsKilled', 'objectivesStolen', 'objectivesStolenAssists', 
            'visionScore','visionWardsBoughtInGame', 'wardsKilled', 'wardsPlaced']

After going through and making sure I will be grabbing everything that I want. I set up the cell below to add up the values and average them out between the team. It worked exactly how I wanted it to so again I added it to my pipeline. 

In [28]:
new = {}
for i in match['info']['participants']:
    if i['teamId'] == 100:
        for j in avg_keys:
            try:
                new['avg'+j[0].upper()+j[1:]] += i[j]
            except:
                new['avg'+j[0].upper()+j[1:]] = i[j]
new.update((k, round(v * 0.2, 3)) for k,v in new.items())
new

{'avgKills': 8.2,
 'avgDeaths': 12.2,
 'avgAssists': 6.6,
 'avgChampLevel': 16.0,
 'avgChampExperience': 15302.8,
 'avgDoubleKills': 1.2,
 'avgTripleKills': 0.0,
 'avgQuadraKills': 0.0,
 'avgPentaKills': 0.0,
 'avgConsumablesPurchased': 3.8,
 'avgDamageDealtToBuildings': 2657.8,
 'avgDamageDealtToObjectives': 9446.4,
 'avgDamageSelfMitigated': 24171.2,
 'avgDetectorWardsPlaced': 0.8,
 'avgGoldEarned': 13615.2,
 'avgGoldSpent': 12931.0,
 'avgInhibitorKills': 0.0,
 'avgItemsPurchased': 23.0,
 'avgLargestKillingSpree': 2.4,
 'avgKillingSprees': 1.6,
 'avgLongestTimeSpentLiving': 430.4,
 'avgMagicDamageDealt': 38503.8,
 'avgMagicDamageDealtToChampions': 7567.6,
 'avgMagicDamageTaken': 4398.8,
 'avgPhysicalDamageDealt': 76610.2,
 'avgPhysicalDamageDealtToChampions': 10587.4,
 'avgPhysicalDamageTaken': 21295.2,
 'avgTotalDamageDealt': 130064.8,
 'avgTotalDamageDealtToChampions': 23560.4,
 'avgTotalDamageShieldedOnTeammates': 0.0,
 'avgTotalDamageTaken': 30428.6,
 'avgTrueDamageDealt': 14950.

While looking through the participants, I found that the challenges was extremely long and has a ton of information. More nit picking to make sure I get all the information I want. 

In [29]:
match['info']['participants'][1]['challenges'].keys()

dict_keys(['12AssistStreakCount', 'abilityUses', 'acesBefore15Minutes', 'alliedJungleMonsterKills', 'baronTakedowns', 'blastConeOppositeOpponentCount', 'bountyGold', 'buffsStolen', 'completeSupportQuestInTime', 'controlWardsPlaced', 'damagePerMinute', 'damageTakenOnTeamPercentage', 'dancedWithRiftHerald', 'deathsByEnemyChamps', 'dodgeSkillShotsSmallWindow', 'doubleAces', 'dragonTakedowns', 'earliestBaron', 'earlyLaningPhaseGoldExpAdvantage', 'effectiveHealAndShielding', 'elderDragonKillsWithOpposingSoul', 'elderDragonMultikills', 'enemyChampionImmobilizations', 'enemyJungleMonsterKills', 'epicMonsterKillsNearEnemyJungler', 'epicMonsterKillsWithin30SecondsOfSpawn', 'epicMonsterSteals', 'epicMonsterStolenWithoutSmite', 'flawlessAces', 'fullTeamTakedown', 'gameLength', 'goldPerMinute', 'hadAfkTeammate', 'hadOpenNexus', 'immobilizeAndKillWithAlly', 'initialBuffCount', 'initialCrabCount', 'jungleCsBefore10Minutes', 'junglerKillsEarlyJungle', 'junglerTakedownsNearDamagedEpicMonster', 'kTurre

I noticed while looking through the challenges, that they are all different lengths. Which means I have my work cut out for me.

In [30]:
first = match['info']['participants'][0]['challenges'].keys()
second = match['info']['participants'][1]['challenges'].keys()
third = match['info']['participants'][2]['challenges'].keys()
new = {}
for k in second:
    new[k] = k in third
for i in range(0,10):
    print(len(match['info']['participants'][i]['challenges'].keys()))

111
109
113
111
110
115
116
113
115
114


To make it a little easier to understand I put it into a DataFrame and called `.info()` again. I need to assign `verbose=True` or it wouldn't display everything I need to see. 

We are working with 16 floats and and 93 ints. I already notice that some of these variables should be booleans and are most likely a 1 or 0 for `True` or `False` respectively, but we will have to verify with our data. 

In [31]:
test_frame = pd.DataFrame([match['info']['participants'][1]['challenges']])
test_frame.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 109 columns):
 #   Column                                  Dtype  
---  ------                                  -----  
 0   12AssistStreakCount                     int64  
 1   abilityUses                             int64  
 2   acesBefore15Minutes                     int64  
 3   alliedJungleMonsterKills                float64
 4   baronTakedowns                          int64  
 5   blastConeOppositeOpponentCount          int64  
 6   bountyGold                              int64  
 7   buffsStolen                             int64  
 8   completeSupportQuestInTime              int64  
 9   controlWardsPlaced                      int64  
 10  damagePerMinute                         float64
 11  damageTakenOnTeamPercentage             float64
 12  dancedWithRiftHerald                    int64  
 13  deathsByEnemyChamps                     int64  
 14  dodgeSkillShotsSmallWindow              int64

I used this to pick which variables I will be using. The list of which is below. I decieded to not use any of the booleans for the team analysis. The `not_used` portion is because not every participant has every variable and I needed to test to see which ones aren't universal. There are a lot more unused variables but these are the ones I wanted to use but can't. 

In [42]:
chal_keys = ['abilityUses', 'acesBefore15Minutes', 'alliedJungleMonsterKills', 'baronTakedowns', 'blastConeOppositeOpponentCount',
             'bountyGold', 'buffsStolen', 'damagePerMinute', 'deathsByEnemyChamps', 
             'dragonTakedowns', 'effectiveHealAndShielding',
             'enemyChampionImmobilizations', 'enemyJungleMonsterKills', 'epicMonsterSteals', 'goldPerMinute',
             'jungleCsBefore10Minutes', 'kda', 'killParticipation', 'killsNearEnemyTurret',
             'killsUnderOwnTurret', 'killsWithHelpFromEpicMonster',  
             'maxKillDeficit', 'multikills', 'outnumberedKills', 'riftHeraldTakedowns',
             'scuttleCrabKills', 'skillshotsDodged', 'skillshotsHit', 'soloKills', 'takedowns',
             'takedownsAfterGainingLevelAdvantage', 'takedownsBeforeJungleMinionSpawn', 
             'turretPlatesTaken', 'turretTakedowns', 'turretsTakenWithRiftHerald', 
             'visionScorePerMinute', 'wardTakedownsBefore20M', 'wardsGuarded']

not_used = [
            'soloTurretsLategame', 'dodgeSkillShotsSmallWindow', 'tookLargeDamageSurvived', 'pickKillWithAlly', 
            'knockEnemyIntoTeamAndKill', 'visionScoreAdvantageLaneOpponent', 'maxLevelLeadLaneOpponent', 
            'laningPhaseGoldExpAdvantage', 'earlyLaningPhaseGoldExpAdvantage', 'earliestBaron', 
            'killsOnLanersEarlyJungleAsJungler', 'junglerKillsEarlyJungle', 'takedownsFirst25Minutes']

This block of code looks eerily familiar, and works almost the exact same way as the one before. And after testing we can see that everything is in order. So I create a function that does the same and I am almost ready for some analysis. 

In [58]:
new = {}
for i in match['info']['participants']:
    if i['teamId'] == 100:
        for j in chal_keys:
                try:
                    new['avg'+j[0].upper()+j[1:]] += i['challenges'][j]
                except:
                    new['avg'+j[0].upper()+j[1:]] = i['challenges'][j]
new.update((k, round(v * 0.2, 4)) for k,v in new.items())
new

{'avgAbilityUses': 153.8,
 'avgAcesBefore15Minutes': 0.0,
 'avgAlliedJungleMonsterKills': 15.76,
 'avgBaronTakedowns': 0.6,
 'avgBlastConeOppositeOpponentCount': 0.2,
 'avgBountyGold': 735.0,
 'avgBuffsStolen': 0.4,
 'avgDamagePerMinute': 741.8184,
 'avgDeathsByEnemyChamps': 12.2,
 'avgDodgeSkillShotsSmallWindow': 44.0,
 'avgDragonTakedowns': 0.0,
 'avgEffectiveHealAndShielding': 43.2,
 'avgEnemyChampionImmobilizations': 13.0,
 'avgEnemyJungleMonsterKills': 1.92,
 'avgEpicMonsterSteals': 0.0,
 'avgGoldPerMinute': 428.6932,
 'avgJungleCsBefore10Minutes': 8.8,
 'avgKda': 1.5386,
 'avgKillParticipation': 0.2426,
 'avgKillsNearEnemyTurret': 1.4,
 'avgKillsUnderOwnTurret': 1.2,
 'avgKillsWithHelpFromEpicMonster': 0.2,
 'avgKnockEnemyIntoTeamAndKill': 0.2,
 'avgMaxKillDeficit': 0.0,
 'avgMultikills': 1.2,
 'avgOutnumberedKills': 2.0,
 'avgPickKillWithAlly': 11.4,
 'avgRiftHeraldTakedowns': 0.2,
 'avgScuttleCrabKills': 0.4,
 'avgSkillshotsDodged': 13.0,
 'avgSkillshotsHit': 27.0,
 'avgSoloKil

In [60]:
match_list = get_matchlist('Zealfire')
data = []
i=0
for match_id in match_list:
    i+=1
    print(i)
    data.append(get_row(match_id))
df = pd.DataFrame(data)
df

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


KeyError: 'challenges'

In [62]:
get_row(match_list[99])

KeyError: 'challenges'

In [64]:
test_match = get_match(match_list[99])

In [67]:
test_match['info']['participants']

[{'assists': 6,
  'baronKills': 0,
  'bountyLevel': 0,
  'champExperience': 9397,
  'champLevel': 12,
  'championId': 92,
  'championName': 'Riven',
  'championTransform': 0,
  'consumablesPurchased': 1,
  'damageDealtToBuildings': 2206,
  'damageDealtToObjectives': 6078,
  'damageDealtToTurrets': 2206,
  'damageSelfMitigated': 16466,
  'deaths': 3,
  'detectorWardsPlaced': 1,
  'doubleKills': 1,
  'dragonKills': 0,
  'firstBloodAssist': False,
  'firstBloodKill': False,
  'firstTowerAssist': False,
  'firstTowerKill': False,
  'gameEndedInEarlySurrender': False,
  'gameEndedInSurrender': False,
  'goldEarned': 8925,
  'goldSpent': 7775,
  'individualPosition': 'TOP',
  'inhibitorKills': 0,
  'inhibitorTakedowns': 0,
  'inhibitorsLost': 0,
  'item0': 6630,
  'item1': 2031,
  'item2': 3158,
  'item3': 3074,
  'item4': 0,
  'item5': 0,
  'item6': 3340,
  'itemsPurchased': 9,
  'killingSprees': 1,
  'kills': 7,
  'lane': 'NONE',
  'largestCriticalStrike': 0,
  'largestKillingSpree': 6,
  

In [68]:
match['info']['participants']

[{'assists': 11,
  'baronKills': 0,
  'bountyLevel': 0,
  'challenges': {'12AssistStreakCount': 0,
   'abilityUses': 208,
   'acesBefore15Minutes': 0,
   'alliedJungleMonsterKills': 2,
   'baronTakedowns': 1,
   'blastConeOppositeOpponentCount': 0,
   'bountyGold': 0,
   'buffsStolen': 0,
   'completeSupportQuestInTime': 0,
   'controlWardTimeCoverageInRiverOrEnemyHalf': 0.18260364271342508,
   'controlWardsPlaced': 2,
   'damagePerMinute': 372.0216369218776,
   'damageTakenOnTeamPercentage': 0.1915849203921449,
   'dancedWithRiftHerald': 0,
   'deathsByEnemyChamps': 8,
   'dodgeSkillShotsSmallWindow': 44,
   'doubleAces': 0,
   'dragonTakedowns': 0,
   'earliestBaron': 1382.9942794950232,
   'earlyLaningPhaseGoldExpAdvantage': -0.19972527027130127,
   'effectiveHealAndShielding': 0,
   'elderDragonKillsWithOpposingSoul': 0,
   'elderDragonMultikills': 0,
   'enemyChampionImmobilizations': 34,
   'enemyJungleMonsterKills': 0,
   'epicMonsterKillsNearEnemyJungler': 0,
   'epicMonsterKil